The objective of the code is to develop a code that shows some of the relevant characteristics of an aquifer (water altitude raster, vadose zone thickness, isopach map, etc.) by having as inputs a point layer with the water depth from the surface and a digital elevation model. The output layers are intended to generate a quick visualization of the aquifer conditions, however, a more in-depth analysis will have to be performed because the code performs an interpolation which is linked to an error percentage.

In [9]:
import arcpy
from arcpy import env
import os
env.overwriteOutput = True

In [10]:
# Set working directory
directoryPath = input("Enter the directory path that contain the input features: ")
env.workspace = directoryPath
wellData = input("Enter name for the points shape file that contains the depth to water: ")
DEM = input("Enter the name for the raster layer that contains the DEM: ")

Enter the directory path that contain the input features:  C:\Users\lvllsrcs\OneDrive - The University of Memphis\1st Semester\Advance GIS\Project\Workspace
Enter name for the points shape file that contains the depth to water:  MRVA_GW_AgencyMeas_2018.shp
Enter the name for the raster layer that contains the DEM:  DEM.tif


In [11]:
fields = arcpy.ListFields(wellData)
print(f"The field names in the layer {wellData} are:")

for field in fields:
    print(field.name, end = ", ")

The field names in the layer MRVA_GW_AgencyMeas_2018.shp are:
FID, Shape, SITE_BADGE, LAT_DD, LONG_DD, LL_DATUM, LL_ACY_CD, LL_METH_CD, ALT_VA, ALT_DATUM, ALT_METHCD, ALT_ACY_VA, ALT_88, URL, NAT_AQFR, STATION_NM, LANDNET, FIPS_ST, MAP_REGION, WELL_DEPTH, LEVCNT2018, DRYCNT2018, FLOCNT2018, WLA88_2018, D2W_2018, DATE_2018, USE_2018, USECMT2018, X_AlbWGS84, Y_ALBWGS84, Meas_type, MeasCat, FIPS_CNTY, CNTY_NAME, 

In [12]:
wellDepthField = input("Enter the field that contains the well depth to water: ")

Enter the field that contains the well depth to water:  WLA88_2018


In [13]:
arcpy.management.CreateFileGDB(directoryPath, "AquiferProperties.gdb")

<Result 'C:\\Users\\lvllsrcs\\OneDrive - The University of Memphis\\1st Semester\\Advance GIS\\Project\\Workspace\\AquiferProperties.gdb'>

In [14]:
arcpy.sa.ExtractValuesToPoints(wellData, DEM, "AquiferProperties.gdb\\welldData_v2")
wellData2 = "AquiferProperties.gdb\\welldData_v2"

In [15]:
arcpy.management.AddFields(wellData2,[["WaterAltitude","DOUBLE","Water altitude"]])

<Result 'C:\\Users\\lvllsrcs\\OneDrive - The University of Memphis\\1st Semester\\Advance GIS\\Project\\Workspace\\AquiferProperties.gdb\\welldData_v2'>

In [16]:
fields = arcpy.ListFields(wellData2)
print(f"The field names in the layer {wellData2} are:")
for field in fields:
    print(field.name, end = ", ")

The field names in the layer AquiferProperties.gdb\welldData_v2 are:
OBJECTID, Shape, SITE_BADGE, LAT_DD, LONG_DD, LL_DATUM, LL_ACY_CD, LL_METH_CD, ALT_VA, ALT_DATUM, ALT_METHCD, ALT_ACY_VA, ALT_88, URL, NAT_AQFR, STATION_NM, LANDNET, FIPS_ST, MAP_REGION, WELL_DEPTH, LEVCNT2018, DRYCNT2018, FLOCNT2018, WLA88_2018, D2W_2018, DATE_2018, USE_2018, USECMT2018, X_AlbWGS84, Y_ALBWGS84, Meas_type, MeasCat, FIPS_CNTY, CNTY_NAME, RASTERVALU, WaterAltitude, 

In [17]:
arcpy.CalculateField_management(in_table=wellData2, field= "WaterAltitude",expression=f"!RASTERVALU! - !{wellDepthField}!",expression_type="PYTHON3")

<Result 'C:\\Users\\lvllsrcs\\OneDrive - The University of Memphis\\1st Semester\\Advance GIS\\Project\\Workspace\\AquiferProperties.gdb\\welldData_v2'>

In [18]:
env.workspace = directoryPath + "\\AquiferProperties.gdb"

In [20]:
cellSize = input("Enter the cell size for the Kriging interpolation: ")
outVarRaster = "VarRaster"
lagSize = 2000
majorRange = 2.6
partialSill = 542
nugget = 0

# Set complex variables
kModelOrdinary = arcpy.sa.KrigingModelOrdinary("CIRCULAR", lagSize,majorRange, partialSill, nugget)
kRadius = arcpy.sa.RadiusFixed(20000, 1)

outKriging = arcpy.sa.Kriging("welldData_v2","WaterAltitude",kModelOrdinary,cellSize,kRadius,outVarRaster)
outKriging.save("Kriging")

Enter the cell size for the Kriging interpolation:  500


In [21]:
env.workspace = directoryPath

The next steps are to calculate the aquifer properties from the generated interpolation and the digital elevation model and standardize the process so that it can be used through a python function.
This code was created using the Arcpy libraries because it was necessary to use a large number of python libraries to be able to generate interpolations through open source.